In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2 
import numpy as np
import pickle
from scipy.misc import imread

from birdseye import BirdsEye
from lanefilter import LaneFilter
from curves import Curves
from helpers import show_images, save_image, roi

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
calibration_data = pickle.load(open("calibration_data.p", "rb" ))

matrix = calibration_data['camera_matrix']
distortion_coef = calibration_data['distortion_coefficient']

source_points = [(580, 460), (205, 720), (1110, 720), (703, 460)]
destination_points = [(320, 0), (320, 720), (960, 720), (960, 0)]

p = { 'sat_thresh': 120, 'light_thresh': 40, 'light_thresh_agr': 205,
      'grad_thresh': (0.7, 1.4), 'mag_thresh': 40, 'x_thresh': 20 }

birdsEye = BirdsEye(source_points, destination_points, matrix, distortion_coef)
laneFilter = LaneFilter(p)
curves = Curves(number_of_windows = 9, margin = 100, minimum_pixels = 50, 
                ym_per_pix = 30 / 720 , xm_per_pix = 3.7 / 700)

In [3]:
def pipeline(img):
    
  ground_img = birdsEye.undistort(img)
  binary = laneFilter.apply(ground_img)
  wb = np.logical_and(birdsEye.sky_view(binary), roi(binary)).astype(np.uint8)
  result = curves.fit(wb)
  ground_img_with_projection = birdsEye.project(ground_img, binary, 
                                result['pixel_left_best_fit_curve'], result['pixel_right_best_fit_curve'])

  text_pos = "vehicle position: " + result['vehicle_position_words']
  text_l = "left radius: " + str(np.round(result['left_radius'], 2)) 
  text_r = " right radius: " + str(np.round(result['right_radius'], 2))    
  cv2.putText(ground_img_with_projection, text_l, (20, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
  cv2.putText(ground_img_with_projection, text_r, (400, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
  cv2.putText(ground_img_with_projection, text_pos, (20, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)

  return ground_img_with_projection

In [4]:
project_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4");
white_clip = clip1.fl_image(pipeline) 
%time white_clip.write_videofile(project_output, audio = False);

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [06:30<00:00,  3.41it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 5min 57s, sys: 1min 12s, total: 7min 9s
Wall time: 6min 31s


In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))